In [1]:
import pysam
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from Bio import SeqIO
import pandas as pd

In [2]:
def aln2cov(samfile, ref_file):
    samfile = pysam.AlignmentFile(samfile, "rb")
    refseq = SeqIO.read(ref_file, "fasta").seq
    coverage = {}
    
    for read in samfile.fetch():
        start_read, end_read = read.reference_start, read.reference_end
        if start_read and end_read: # Check mapped
            for pos in range(start_read, end_read):
                if pos in coverage:
                    coverage[pos] += 1
                else:
                    coverage[pos] = 1
    samfile.close()
    return coverage

def mean_coverage(dict_coverage):
    return np.array(list(dict_coverage.values())).mean()

def count_coverage(dict_coverage, refrence_sequance):
    return round(len(dict_coverage.keys()) / len(refrence_sequance), 15)

def plot_coverage(dict_coverage):
    x, y = list(dict_coverage.keys()), list(dict_coverage.values())
    plt.figure(figsize=(15, 10))
    plt.xlabel('Позиция нуклеотида в геноме')
    plt.ylabel('Количество ридов, которые покрыли нуклеотид')
    plt.axhline(y=mean_coverage(dict_coverage), label='Среднее покрытие', color='r', linestyle='--')
    plt.legend()
    sns.scatterplot(x, y)

def count_indels(samfile):
    samfile = pysam.AlignmentFile(samfile, "r") 
    insertions, deletions, counter = 0, 0, 0
    
    for x in samfile.fetch():
        if x.cigartuples:
            for type_indel, lenght in x.cigartuples:
                if type_indel == 0:
                    counter += lenght
                if type_indel == 1:
                    insertions += lenght
                if type_indel == 2:
                    deletions += lenght
    return (insertions / counter, deletions / counter)
    
def plot_indel(samfile):
    samfile = pysam.AlignmentFile(samfile, "r") 
    lenght_insertion = []
    lenght_deletion = []
    lenght_indel = []
    for x in samfile.fetch():
        if x.cigartuples:
            for type_indel, lenght in x.cigartuples:
                if type_indel == 1:
                    lenght_insertion.append(lenght)
                if type_indel == 2:
                    lenght_deletion.append(lenght)
                if type_indel == 1 or type_indel == 2:
                    lenght_indel.append(lenght)

    tempory_counters = [Counter(array) for array in [lenght_insertion, lenght_deletion]]
    
    fig, ax = plt.subplots(figsize=(15, 10))
    for counter, label in zip(tempory_counters, ['insertion', 'deletion']):
        x = list(counter.keys())
        y = list(counter.values())
        plt.scatter(x, y, label=label, linewidth=2)

    plt.legend(loc=1, prop={'size': 16})
    plt.xlabel('Длина индела')
    plt.ylabel('Количество инделов с такой длиной')
    plt.grid()
    plt.xlim(0, 100)

def plot_kde_replacment(samfile):
    samfile = pysam.AlignmentFile(samfile, "r") 
    quality_indel = []
    for read in samfile.fetch():
        acnt, generator = -1, read.cigartuples
        if generator:
            for ind, cnt in generator:
                if ind == 0:
                    acnt += cnt
                if ind == 1:
                    quality_indel.append(ord(read.qual[acnt]) - 33)
    
    plt.figure(figsize=(15, 10))
    plt.xlabel('Качество нуклеотида')
    plt.ylabel('Частота')
    sns.kdeplot(quality_indel)
def template_lenght(samfile, confidience, scale=False):
    samfile = pysam.AlignmentFile(samfile, "r") 
    templates = []
    for x in samfile.fetch():
        if x.template_length > 0:
            templates.append(x.template_length)
            
    samfile.close()
    
    counter_templates = Counter(templates)
    x = np.array(list(range(len(counter_templates))))
    y = np.array([counter_templates[i] for i in range(len(counter_templates))])
    if scale:
        x, y = x[:1000], y[:1000]
        
    #draw temlates lenght
    plt.figure(figsize=(15, 10))
    plt.xlabel('Расстояние вставки')
    plt.ylabel('Количество с таким расстоянием вставки')
    plt.grid()
    
    # Calculate cond interval
    cnt, min_index_l, min_index_r = 0, 0, 0
    left, right = sum(y) * (1 - confidience) / 2, sum(y) * (1 + confidience) / 2
    for index, elem in enumerate(array[1]):
        cnt += elem
        if cnt > left and min_index_l == 0:
            min_index_l = index
        if cnt > right and min_index_r == 0:
            min_index_r = index
            break
    
    #draw other
    plt.axvline(x[min_index_l], linestyle='--', color='r', label='left confidence interval')
    plt.axvline(x[min_index_r], linestyle='--', color='r', label='right confidence interval')
    plt.legend()
    sns.scatterplot(x, y, color='k')
    return x, y

def matrix_substitutions(samfile, refrence):
    samfile = pysam.AlignmentFile(samfile, "rb")
    replacment = {'A' : [], 'G' : [], 'C' : [], 'T' : [], '_' : []}
    refseq = SeqIO.read(refrence, 'fasta')
    
    for read in samfile.fetch():
        generator, acc = [], 0
        if read.cigartuples and read.seq:
            for ind, count in read.cigartuples:
                generator.append([ind, [acc, acc + count]])
                acc += count
            l_read, l_reference, counter_ind, cnt = 0, read.reference_start, 0, 0
            iterations = 0
            while l_read != len(read.seq):
                if generator[counter_ind][0] == 0:
                    replacment[refseq[l_reference]].append(read.seq[l_read])
                    l_read += 1
                    l_reference += 1
                if generator[counter_ind][0] == 1:
                    replacment['_'].append(read.seq[l_read])
                    l_reference += 1
                    l_read += 1
                if generator[counter_ind][0] == 2:
                    replacment[refseq[l_reference]].append('_')
                    l_read += 1
                if generator[counter_ind][0] == 4:
                    l_read += 1
                    l_reference += 1
                if l_read == generator[counter_ind][1][1]:
                    counter_ind += 1
                if iterations > 150:
                    break
    samfile.close()
    
    return replacment

def matrix_substitutions_2(samfile, refrence):
    samfile = pysam.AlignmentFile(samfile, threads=4)
    refseq = SeqIO.read(refrence, 'fasta')
    matrix = {nucl_1 : {nucl_2 : 0 for nucl_2 in ['A', 'G', 'C', 'T', '']} for nucl_1 in ['A', 'G', 'C', 'T']}
    for columns in samfile.pileup():
        ref_nucl = refseq[columns.reference_pos]
        for nucl in ['A', 'C', 'T', 'G', '']:
            matrix[ref_nucl][nucl] += Counter(columns.get_query_sequences())[nucl]
    return matrix

In [4]:
BAM_FILE_RAW = 'aln_sorted.bam'
BAM_FILE_TRIMMOMATIC = 'aln_sorted_tr.bam'
REFERENCE_FILE = 'MG1655-K12.first400K.fasta'

In [67]:
bamfile_raw = pysam.AlignmentFile(BAM_FILE_RAW, threads=4)
bamfile_trimmomatic = pysam.AlignmentFile(BAM_FILE_TRIMMOMATIC, threads=4)
refseq = SeqIO.read(REFERENCE_FILE, 'fasta')
count = 0
matrix_replacment = {'Undetected' : 0, 'Falsely corrected' : 0, 'Detected corrected' : 0, 
                     'Correctly unmodified' : 0, 'Detected and removed' : 0, 'Incorrectly removed' : 0}
for columns_1, columns_2 in zip(bamfile_raw.pileup(), bamfile_trimmomatic.pileup()):
    ref_nucl = refseq[columns_1.reference_pos]
    names_reads = {}
    
    for read_corr in columns_2.pileups:
        if read_corr.query_position is not None:
            names_reads[read_corr.alignment.query_name] = read_corr.alignment.query_sequence[read_corr.query_position]
        else:
            names_reads[read_corr.alignment.query_name] = ''
    
    for read_raw in columns_1.pileups:
        if read_raw.alignment.query_name in names_reads:
            if read_raw.query_position:
                raw_nucl  = read_raw.alignment.query_sequence[read_raw.query_position]
                corr_nucl = names_reads[read_raw.alignment.query_name]
                if (raw_nucl == corr_nucl) and corr_nucl != ref_nucl:
                    matrix_replacment['Undetected'] += 1

                if (raw_nucl == ref_nucl) and corr_nucl != ref_nucl:
                    matrix_replacment['Falsely corrected'] += 1

                if (raw_nucl != ref_nucl) and corr_nucl == ref_nucl:
                    matrix_replacment['Detected corrected'] += 1

                if raw_nucl == ref_nucl and corr_nucl == ref_nucl:
                    matrix_replacment['Correctly unmodified'] += 1
        else:
            if read_raw.query_position:
                raw_nucl  = read_raw.alignment.query_sequence[read_raw.query_position]
                if raw_nucl == ref_nucl:
                    matrix_replacment['Incorrectly removed'] += 1
                else:
                    matrix_replacment['Detected and removed'] += 1
    count += 1
    if count % 10000 == 0:
        print(count)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000


In [68]:
matrix_replacment

{'Undetected': 421002,
 'Falsely corrected': 155,
 'Detected corrected': 219,
 'Correctly unmodified': 239846944,
 'Detected and removed': 22007,
 'Incorrectly removed': 6582739}